<a href="https://colab.research.google.com/github/AppleTater/Jinkerson-Lab-Research/blob/master/New_Cumulative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Files that need to be uploaded**


---
* TAGData.csv
* large-lib_full_2016.10.25.csv



# Import libraries

In [ ]:
%reload_ext rpy2.ipython

# %%R
# library(ggplot2)

import rpy2.robjects as robjects                          # common rpy2 objects under the abbreviation 'robjects'
from rpy2.robjects.packages import importr                # rpy2 packages under the abbreviation 'importr'
from rpy2.robjects.vectors import FloatVector             # rpy2 for float conversion
import gc                                                 # rpy2 library
R_stats = importr('stats',  robject_translations={'format_perc': '_format_perc'})                                
# rpy2 stats

from io import StringIO

from rpy2.rinterface_lib.sexp import StrSexpVector        # rpy2 interface
from functools import partial                             # rpy2 tools under the abbreviation 'partial'
from rpy2.ipython import html                             # html support
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

from rpy2.robjects.conversion import localconverter       # rpy2 converter under the abbreviation 'localconverter'  

import pandas as pd                                       # these libraries are for redundancy
import rpy2.robjects as ro                                # ↓
from rpy2.robjects.packages import importr                # ↓
from rpy2.robjects import pandas2ri                       # ↓

In [ ]:
import pandas as pd                                       # importing common pandas libraries under the abbreviation 'pd' 
import matplotlib.pyplot as plt                           # was for the requested histogram under the abbreviation 'plt' 

import numpy.matlib                                       # to calculate mean and median
import numpy as np                                        # numpy under the abbreviation 'np' 
from numpy import median                                  # from the numpy library, import how to get the median

import scipy.stats as stats                               # libraries used for BH adjustment under the abbreviation 'stats' 
import scipy.special                                      # special libraries from scipy

import re                                                 # more libraries
import sys                                                # more libraries

from IPython.display import (                             # display data, if wanted, as different media file types
    display_pretty, display_html, display_jpeg,
    display_png, display_json, display_latex, display_svg
)

from IPython.display import display                       # more display tools
import itertools                                          # more display tools
import gc                                                 # more display tools

import statsmodels.api as sm                              # stats models under the abbreviation 'sm'
import statsmodels.stats.multitest as smm                 # stats models under the abbreviation 'smm'
import statsmodels.formula.api as smf                     # stats models under the abbreviation 'smf'

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



# Run
---


* Given gene name, find data (name → data)
* Given data, find name(s) (data → name)
* Change list values, let run
* Choose bin, see mutants in bin
* Given data, find binned genes' p-value





# Given name, find data


In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv'

cre_name = input("Cre name: ")                                                  # user input for specific cre
cre_name_v5 = ".v5.5"                                                           # store value ".v5.5" as cre_name_v5
solo_name = cre_name                                                            # solo_name stores the inputted cre
cre_name += cre_name_v5                                                         # cre_name strores the cre name with .v5.5 appended

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
ratio = float(input("'Hits' are ratios that are greater than: "))               # user input for ratio 
#------------------------------------------------------------------------------------------------------------------------------------

# ↓ Gene specific ↓

# for 'TAG Data.csv'
cre_tag_df = tag[['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads' ]]
cre_tag_only = pd.DataFrame()
cre_tag_only = cre_tag_df.loc[((cre_tag_df['gene'] == cre_name) | (cre_tag_df['gene'] == solo_name))]
# keep only the listed columns

# for 'large-lib_full_2016.10.25.csv'
cre_lib_df = lib[['gene', 'IB', 'confidence_level']]
cre_lib_only = cre_lib_df.loc[((cre_lib_df['gene'] == cre_name) | (cre_lib_df['gene'] == solo_name))]
# keep only the listed columns

merged_cre = cre_tag_only.merge(cre_lib_only, on = "IB")                        # merge data from lib and tag based on their IB
cre_count = merged_cre['gene_x'].value_counts()                                 # store the number of times the specified gene is in merged_cre, stored as cre_count

merged_cre['Ratios: HLG / ALL (Whole)'] = merged_cre['R5-S#11-HLG_WHOLE_normalized_reads'] / merged_cre['R5-S#11-ALL_WHOLE_normalized_reads']       
# calculate HLG / ALL for ratio
whole = merged_cre
#merged_cre = merged_cre.replace([np.inf, -np.inf], np.nan)                     # where there is a value equal to 'inf', it is replaced by 'NaN' 

merged_cre = merged_cre[merged_cre.confidence_level <= confidence]              # restricts merged_cre to rows that have confidence levels greater than inputted confidence

merged_cre = merged_cre.drop(columns = {'gene_y'}) #'R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged_cre = merged_cre.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns and reset index

merged_cre["RatioType"] = np.where(merged_cre["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged_cre, 'RatioType' to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged_cre = merged_cre.dropna().reset_index(drop = True)    
#merged_cre = merged_cre.reset_index(drop = True)
# drop rows with Nan and reset index

# ↓ Not gene specific ↓

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
# reorder merged to match the column order of merged_cre
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Counts total number of specific gene hits and specific gene non-hits based on ratio cutoff
given = pd.DataFrame(data = merged_cre, columns = ['Gene'])                     # dataframe based on merged_cre, only gene names
given = given['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
given = given.sort_values('Gene').reset_index(drop = True)                      # sort dataframe by gene name, reset index

given['Hit'] = len(merged_cre[merged_cre.RatioType == 'Hit'])                   # count number of 'Hit' in RatioType from merged_cre
given['Non-Hit'] = len(merged_cre[merged_cre.RatioType == "Non-Hit"])           # count number of 'Non-Hit' in RatioType from merged_cre

# Counts of all unique genes, counts of hits and non-hits based on ratio cutoff
counts = pd.DataFrame(data = clean['Gene'], columns = ['Gene'])                 # dataframe based on clean, only gene names
counts = counts['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
counts = counts.sort_values('Gene').reset_index(drop = True)                    # sort dataframe by gene name, reset index

counts['Hit'] = len(clean[clean.RatioType == 'Hit'])                            # count the number of 'Hit' in RatioType from clean
counts['Non-Hit'] = len(clean[clean.RatioType == "Non-Hit"])                    # count the number of 'Non-Hit' in RatioType from clean

counts = counts[counts['Count'] >= count].reset_index(drop = True)              # drops rows that have a count less than the inputted count, rest index
specific = counts.loc[counts['Gene'] == solo_name]                              # specific becomes the row that contains the specified cre name

frames = [given, specific]                                                      # creates an array(?) that are the given and specific dataframes

p_val_calc = pd.concat(frames).drop(columns = {'Count'}).reset_index(drop = True)
# concatenate the dataframes in frames, drop the 'Count' column, reset index
p_val_calc.iloc[1,0] = 'Total'                                                  # change value in the contingency table from the gene name to 'Total'
 # contingency table:                          
        # |            |   Hit   |   Non-Hit   |
        # |————————————|—————————|—————————————|  
        # |  cre_name  |         |             |
        # |————————————|—————————|—————————————|  
        # |    Total   |         |             |
p_val_calc = p_val_calc.set_index('Gene')                                       # make the 'Gene' column to be the index

oddsratio, pvalue = stats.fisher_exact(p_val_calc)                              # calculate p-value
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

display (p_val_calc)
display(merged_cre)                                                             # display dataframe for the specifed gene (Gene, IB, Confidence Level, Ratio, RatioType)
print(cre_count)                                                                # print all isntances of the specified gene count prior to dropping rows
print(pvalue)                                                                   # print p-value


NameError: ignored

# Given data, find name(s)

In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
ratio = float(input("'Hits' are ratios that are greater than: "))               # user input for ratio 
bh_num = float(input("Enter the value that all adjusted p-values should be less than (0-1): "))
# user input for the maximum of the adjusted p-value
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
# reorder merged 
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# total counts for all genes based on ratio (Hit vs Non-Hit counts)
total_counts = clean['RatioType'].value_counts()                                # RatioType column is counted hit and non-hit
total_counts = pd.DataFrame(total_counts)                                       # makes into a dataframe
total_counts = total_counts.reset_index()                                       # reset the index
total_counts = total_counts.rename(columns = {'index' : 'Gene', 'RatioType' : 'Hit'})
# rename columns
total_counts['Non-Hit'] = ""                                                    # make empty column
total_counts.iloc[0,0] = 'Total'                                                # replace value in cell (0,0) to 'Total'
total_counts.iloc[0,2] = total_counts.iloc[0,1]                                 # replace value in cell (0,2) with value in (0,1)
total_counts.iloc[0,1] = total_counts.iloc[1,1]                                 # replace value in cell (0,1) with value in (1,1)
total_counts = total_counts.drop([1])                                           # drop last row
total_counts = total_counts.set_index('Gene')                                   # make index of the dataframe to be the 'Gene' column
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

hit_type = clean[['Gene', 'RatioType', 'Count']]                                # new dataframe, takes only 'Gene' 'RatioType', and 'Count' from clean
unique_gene = pd.DataFrame(hit_type.Gene.unique(), columns = {'Gene'})          # new dataframe, with only unique gene names
unique_gene['Hit'] = ""                                                         # empty column in unique_gene
unique_gene['Non-Hit'] = ""                                                     # empty column in unique_gene

pd.options.mode.chained_assignment = None  # default='warn'                     # remove warning messages

# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library:
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

super_x = []                                                                    # empty list

for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = hit_type.loc[hit_type.Gene == unique_gene.iloc[i,0]]              # splice hit_type by the values of unique_gene (gene names)
  num_hit = len(same_gene[(same_gene['RatioType'] == 'Hit')])                   # num_hit is the number of times 'Hit' is counted in the RatioType column of same_gene
  num_non_hit = len(same_gene[(same_gene['RatioType'] == 'Non-Hit')])           # num_non_hit is the number of times 'Non-Hit' is counted in the RatioType column of same_gene
  same_gene['Hit'] = num_hit                                                    # new column in same_gene, stores the number of hits
  same_gene['Non-Hit'] = num_non_hit                                            # new column in same_gene, stores the number of non-hits
  same_gene = same_gene.drop_duplicates(subset = 'Gene', keep = "first").drop(columns = {'RatioType', 'Count'}).set_index('Gene')
  # keep only first row, drop columns, make 'Gene' column index
  frames = [same_gene, total_counts]                                            # store same_gene and total_counts as a 2 x 2 matrix
  contingency_table = pd.concat(frames)                                         # make the 2 x 2 matrix into a dataframe with a 2 x 2 dimension
  oddsratio, pvalue = stats.fisher_exact(contingency_table)                     # run the fisher exact test on the contingency table, find p-value
  contingency_table ['p-value'] = pvalue                                        # store the p-value in a new column in contingency_table
  contingency_table = contingency_table.drop(['Total'])                         # drop Total
  
  super_x.append(contingency_table)                                             # grow super_x list


super_x = pd.concat(super_x, axis = 0)                                          # concatenate super_x and make it into a dataframe
super_x["Adjusted p-value"] = FDR_adjust_pvalues(super_x['p-value'])
# run FDR_adjust_pvalues on the p-value column, and store new value in "Adjusted p-value" column
super_x = super_x.loc[super_x["Adjusted p-value"] < bh_num]                     # show genes that have an adjusted p-value that is less than bh_num 

display(super_x, total_counts)                        


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Drop data if the total counts of a gene is less than: 20
Drop data if the confidence level is greater than: 1
'Hits' are ratios that are greater than: 0.19
Enter the value that all adjusted p-values should be less than (0-1): 0.5


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre01.g018900,16,12,0.008028,0.439231
Cre04.g217935,5,37,0.002824,0.360471
Cre04.g229550,17,10,0.001491,0.321656
Cre06.g268800,17,12,0.004646,0.402926
Cre06.g284850,14,9,0.006312,0.402926
Cre12.g556150,14,9,0.006312,0.402926
Cre13.g589400,14,7,0.001680,0.321656


,Hit,Non-Hit
Gene,,
Total,5126,10612


# Given data, find name(s)
**With Dictionary**


In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
ratio = float(input("'Hits' are ratios that are greater than: "))               # user input for ratio 
bh_num = float(input("Enter the value that all adjusted p-values should be less than (0-1): "))
# user input for the maximum of the adjusted p-value
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
# reorder merged 
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# total counts for all genes based on ratio (Hit vs Non-Hit counts)
total_counts = clean['RatioType'].value_counts()                                # RatioType column is counted hit and non-hit
total_counts = pd.DataFrame(total_counts)                                       # makes into a dataframe
total_counts = total_counts.reset_index()                                       # reset the index
total_counts = total_counts.rename(columns = {'index' : 'Gene', 'RatioType' : 'Hit'})
# rename columns
total_counts['Non-Hit'] = ""                                                    # make empty column
total_counts.iloc[0,0] = 'Total'                                                # replace value in cell (0,0) to 'Total'
total_counts.iloc[0,2] = total_counts.iloc[0,1]                                 # replace value in cell (0,2) with value in (0,1)
total_counts.iloc[0,1] = total_counts.iloc[1,1]                                 # replace value in cell (0,1) with value in (1,1)
total_counts = total_counts.drop([1])                                           # drop last row
total_counts = total_counts.set_index('Gene')                                   # make index of the dataframe to be the 'Gene' column
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

hit_type = clean[['Gene', 'RatioType', 'Count']]                                # new dataframe, takes only 'Gene' 'RatioType', and 'Count' from clean
unique_gene = pd.DataFrame(hit_type.Gene.unique(), columns = {'Gene'})          # new dataframe, with only unique gene names
unique_gene['Hit'] = ""                                                         # empty column in unique_gene
unique_gene['Non-Hit'] = ""                                                     # empty column in unique_gene

pd.options.mode.chained_assignment = None  # default='warn'                     # remove warning messages

# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library:
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

super_x = []                                                                    # empty list
pval_dict = {}                                                                  # empty dictionary

for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = hit_type.loc[hit_type.Gene == unique_gene.iloc[i,0]]              # splice hit_type by the values of unique_gene (gene names)
  num_hit = len(same_gene[(same_gene['RatioType'] == 'Hit')])                   # num_hit is the number of times 'Hit' is counted in the RatioType column of same_gene
  num_non_hit = len(same_gene[(same_gene['RatioType'] == 'Non-Hit')])           # num_non_hit is the number of times 'Non-Hit' is counted in the RatioType column of same_gene
  same_gene['Hit'] = num_hit                                                    # new column in same_gene, stores the number of hits
  same_gene['Non-Hit'] = num_non_hit                                            # new column in same_gene, stores the number of non-hits
  same_gene = same_gene.drop_duplicates(subset = 'Gene', keep = "first").drop(columns = {'RatioType', 'Count'}).set_index('Gene')

  hit_or_not = same_gene[['Hit', 'Non-Hit']].to_numpy()                         # set hit_or_not to be same_gene columns of 'Hit' and 'Non-Hit' to by a numpy array
  hit_or_not = tuple(map(tuple, hit_or_not))                                    # maps hit_or_not to be a tuple, like a coordinate pair

  if (hit_or_not not in pval_dict):                                             # if statement for if the pair is in pval_dict, if it is not in the dictionary, then do the rest
    frames = [same_gene, total_counts]                                          # store same_gene and total_counts as a 2 x 2 matrix
    contingency_table = pd.concat(frames)                                       # make the 2 x 2 matrix into a dataframe with a 2 x 2 dimension
    oddsratio, pvalue = stats.fisher_exact(contingency_table)                   # run the fisher exact test on the contingency table, find p-value
    contingency_table ['p-value'] = pvalue                                      # store the p-value in a new column in contingency_table
    pval_dict = {hit_or_not : contingency_table['p-value']}                     # store the p-value in the dictionary to be located by the pair
# end of if

  elif (hit_or_not in pval_dict):                                               # flip side of the if statement, if the pair is in the dictionary
    contingency_table['p-value'] = pval_dict[hit_or_not]                        # then set the contingency_table column, 'p-value' to be the value located by the pair, stored in the dictionary
# end of else if

  contingency_table = contingency_table.drop(['Total'])                         # drop Total

  super_x.append(contingency_table)                                             # grow super_x list


super_x = pd.concat(super_x, axis = 0)                                          # concatenate super_x and make it into a dataframe
super_x["Adjusted p-value"] = FDR_adjust_pvalues(super_x['p-value'])
# run FDR_adjust_pvalues on the p-value column, and store new value in "Adjusted p-value" column
super_x = super_x.loc[super_x["Adjusted p-value"] < bh_num]                     # show genes that have an adjusted p-value that is less than bh_num
display(super_x, total_counts)                        



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Drop data if the total counts of a gene is less than: 20
Drop data if the confidence level is greater than: 1
'Hits' are ratios that are greater than: 0.19
Enter the value that all adjusted p-values should be less than (0-1): 0.5


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre01.g018900,16,12,0.008028,0.439231
Cre04.g217935,5,37,0.002824,0.360471
Cre04.g229550,17,10,0.001491,0.321656
Cre06.g268800,17,12,0.004646,0.402926
Cre06.g284850,14,9,0.006312,0.402926
Cre12.g556150,14,9,0.006312,0.402926
Cre13.g589400,14,7,0.001680,0.321656


,Hit,Non-Hit
Gene,,
Total,5126,10612


# List adjustment and running

In [ ]:
# CHANGE LISTS
l_count = [10,12,14,16]
l_confidence = [1,2,3]
l_ratio = [.6,.7,.8,1,2]
l_bh_num = [.3]

In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

#------------------------------------------------------------------------------------------------------------------------------------
# start for loop here:
for count in l_count:
  for confidence in l_confidence:
    for ratio in l_ratio:
      for bh_num in l_bh_num:
        # for 'TAG Data.csv'
        tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
        # keep only the listed columns
        tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
        # drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
        tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
        tag_df.reset_index(drop = True)                                                 # reset index

        # for 'large-lib_full_2016.10.25.csv'
        lib_df = lib[['gene', 'IB', 'confidence_level' ]]
        # keep only the listed columns
        lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
        # drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
        lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
        lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
        lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
        merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
        # calculate HLG / ALL for ratio      
        merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
        # drop listed columns
        merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
        # rename columns
        merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
        # new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
        merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
        # reorder merged 
        merged = merged.sort_values('Gene')
        # sort the values by the gene name
        merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
        # replace inf by Nan, drop rows that has Nan, reset the index

        merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
        merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
        # count the number of instances of each gene, stored in a new column, 'Count'
        merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
        # drop rows that has a count that is less than the inputted count restriction
        merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
        # drop rows that have Nan, sort dataframe by gene name, reset index

        clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        # total counts for all genes based on ratio (Hit vs Non-Hit counts)
        total_counts = clean['RatioType'].value_counts()                                # RatioType column is counted hit and non-hit
        total_counts = pd.DataFrame(total_counts)                                       # makes into a dataframe
        total_counts = total_counts.reset_index()                                       # reset the index
        total_counts = total_counts.rename(columns = {'index' : 'Gene', 'RatioType' : 'Hit'})
        # rename columns
        total_counts['Non-Hit'] = ""                                                    # make empty column
        total_counts.iloc[0,0] = 'Total'                                                # replace value in cell (0,0) to 'Total'
        total_counts.iloc[0,2] = total_counts.iloc[0,1]                                 # replace value in cell (0,2) with value in (0,1)
        total_counts.iloc[0,1] = total_counts.iloc[1,1]                                 # replace value in cell (0,1) with value in (1,1)
        total_counts = total_counts.drop([1])                                           # drop last row
        total_counts = total_counts.set_index('Gene')                                   # make index of the dataframe to be the 'Gene' column
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        hit_type = clean[['Gene', 'RatioType', 'Count']]                                # new dataframe, takes only 'Gene' 'RatioType', and 'Count' from clean
        unique_gene = pd.DataFrame(hit_type.Gene.unique(), columns = {'Gene'})          # new dataframe, with only unique gene names
        unique_gene['Hit'] = ""                                                         # empty column in unique_gene
        unique_gene['Non-Hit'] = ""                                                     # empty column in unique_gene

        pd.options.mode.chained_assignment = None  # default='warn'                     # remove warning messages

        # ↓ Benjamini-Hochberg Adjustment of p-values ↓
        # calls on a statisical adjustment from an R library:
        def FDR_adjust_pvalues(case, N=None, method='BH'):
            """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

            N and method are passed to R_stats.p_adjust:
            - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
            - method is the name of the adjustment method to use (inherited from R)

            Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
            trying to do it on single p-values, even with adjusted N, will give different results!
            """
            if not method in R_stats.p_adjust_methods:
                raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
            if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
            else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

        super_x = []                                                                    # empty list

        for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
          same_gene = hit_type.loc[hit_type.Gene == unique_gene.iloc[i,0]]              # splice hit_type by the values of unique_gene (gene names)
          num_hit = len(same_gene[(same_gene['RatioType'] == 'Hit')])                   # num_hit is the number of times 'Hit' is counted in the RatioType column of same_gene
          num_non_hit = len(same_gene[(same_gene['RatioType'] == 'Non-Hit')])           # num_non_hit is the number of times 'Non-Hit' is counted in the RatioType column of same_gene
          same_gene['Hit'] = num_hit                                                    # new column in same_gene, stores the number of hits
          same_gene['Non-Hit'] = num_non_hit                                            # new column in same_gene, stores the number of non-hits
          same_gene = same_gene.drop_duplicates(subset = 'Gene', keep = "first").drop(columns = {'RatioType', 'Count'}).set_index('Gene')
          # keep only first row, drop columns, make 'Gene' column index
          frames = [same_gene, total_counts]                                            # store same_gene and total_counts as a 2 x 2 matrix
          contingency_table = pd.concat(frames)                                         # make the 2 x 2 matrix into a dataframe with a 2 x 2 dimension
          oddsratio, pvalue = stats.fisher_exact(contingency_table)                     # run the fisher exact test on the contingency table, find p-value
          contingency_table ['p-value'] = pvalue                                        # store the p-value in a new column in contingency_table
          contingency_table = contingency_table.drop(['Total'])                         # drop Total
          
          super_x.append(contingency_table)                                             # grow super_x list


        super_x = pd.concat(super_x, axis = 0)                                          # concatenate super_x and make it into a dataframe
        super_x["Adjusted p-value"] = FDR_adjust_pvalues(super_x['p-value'])
        # run FDR_adjust_pvalues on the p-value column, and store new value in "Adjusted p-value" column
        num_bh_adj = super_x.loc[super_x["Adjusted p-value"] < bh_num].shape[0]                     # show genes that have an adjusted p-value that is less than bh_num  
        # pd.set_option("display.max_rows", num_bh_adj)
        super_x = super_x.loc[super_x["Adjusted p-value"] < bh_num]
        
        print ("Count: ", count, " | ", "Confidence: ", confidence, " | ", "Ratio: ", ratio, " | ", "Adjusted p-value", bh_num, " | ", "Number of significant genes: ", num_bh_adj, " | ", "Data: ", super_x.reset_index().values.tolist())
        # display(super_x.sort_values("Adjusted p-value"), total_counts)                        


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Count:  10  |  Confidence:  1  |  Ratio:  0.6  |  Adjusted p-value 0.3  |  Number of significant genes:  5  |  Data:  [['Cre03.g174250', 9, 2, 0.00056237414971118, 0.2300110272318726], ['Cre07.g330200', 13, 4, 9.960233592141e-05, 0.10184338847964174], ['Cre10.g433300', 9, 2, 0.00056237414971118, 0.2300110272318726], ['Cre13.g584901', 14, 2, 2.8580375147632393e-06, 0.005844686717690824], ['Cre13.g589400', 14, 7, 0.0005464746751730446, 0.2300110272318726]]
Count:  10  |  Confidence:  1  |  Ratio:  0.7  |  Adjusted p-value 0.3  |  Number of significant genes:  2  |  Data:  [['Cre07.g330200', 13, 4, 7.534886591125829e-05, 0.1540884307885232], ['Cre13.g584901', 12, 4, 0.00019920338436280616, 0.2036854605109693]]
Count:  10  |  Confidence:  1  |  Ratio:  0.8  |  Adjusted p-value 0.3  |  Number of significant genes:  3  |  Data:  [['Cre07.g330200', 13, 4, 5.3504703024739885e-05, 0.10941711768559306], ['Cre10.g433300', 9, 2, 0.0003596398549698382, 0.245154501137773], ['Cre13.g584901', 12, 4, 0

# Bin p-value



In [ ]:
import numpy as np
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()

stats = importr('stats')

tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

number_of_bins = int(input("Number of bins: "))                                 # user input for number of bins
count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
range_dict = {}

#print("1. Ratio of 0.0 ≤ x ≤ 0.5\n2. Ratio of 0.5 < x ≤ 1.0\n3. Ratio of 1.0 < x ≤ 2.0\n4. Ratio of 2.0+")
#!
# user input for choice of bin
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["Bin Type"] = ""
1
print("\nEnter the range signs as you would reading ranges left to right, keep in mind possible overlapping ranges")
for i in range(number_of_bins):
  increased_i = i + 1
  print("Bin Number", i + 1,":")

  if (i == number_of_bins - 1):
    bin_min = float(input("\tMinimum: "))
    min_sign = input("Enter sign for the inequality: ")
    if (min_sign == "<="):
      print("x ≤" , bin_min)
      bin_range = ("x <= " + str(bin_min))
    elif (min_sign == "<"):
      print("x <" , bin_min)
      bin_range = ("x < " + str(bin_min))
    range_dict['bin_range_' + str(increased_i)] = bin_range
    locals().update(range_dict)
    break

  if (i == 0):
    bin_min = float(input("\tMinimum: "))
    bin_max = float(input("\tMaximum: "))
    max_sign = input("Enter sign for the right-hand side of the inequality: ")
    if (max_sign == "<="):
      print(bin_min , "≤ x ≤" , bin_max)
      bin_range = (str(bin_min) + " <= x <= " + str(bin_max))
    elif (max_sign == "<"):
      print(bin_min , "≤ x <" , bin_max)
      bin_range = (str(bin_min) + " <= x < " + str(bin_max))

  else:
    bin_min = float(input("\tMinimum: "))
    bin_max = float(input("\tMaximum: "))

    min_sign = input("Enter sign for the left-hand side of the inequality: ")
    max_sign = input("Enter sign for the right-hand side of the inequality: ")

    if ((min_sign == "<=") & (max_sign == "<=")):
      print(bin_min , "≤ x ≤" , bin_max)
      bin_range = (str(bin_min) + " <= x <= " + str(bin_max))
    elif ((min_sign == "<") & (max_sign == "<=")):
      print(bin_min , "< x ≤" , bin_max)
      bin_range = (str(bin_min) + " < x <= " + str(bin_max))
    elif ((min_sign == "<=") & (max_sign == "<")):
      print(bin_min , "≤ x <" , bin_max) 
      bin_range = (str(bin_min) + " <= x < " + str(bin_max))    
    elif ((min_sign == "<") & (max_sign == "<")):
      print(bin_min , "< x <" , bin_max)
      bin_range = (str(bin_min) + " < x < " + str(bin_max))
    

  range_dict['bin_range_' + str(increased_i)] = bin_range
  locals().update(range_dict)
    
print('\n')
for key, value in range_dict.items():
  print(key, " : " , value)

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

string_ranges = str(range_dict.values())
list_ranges = re.findall("\d+\.\d+", string_ranges)

range_frame = pd.DataFrame(list_ranges, columns = ["Value"])
range_frame_even = range_frame[range_frame.index % 2 == 0].reset_index(drop = True)
range_frame_odd = range_frame[range_frame.index % 2 == 1].reset_index(drop = True)

range_frame = pd.concat([range_frame_even, range_frame_odd], axis = 1)
range_frame.columns = ['Min', 'Max']
range_frame = range_frame.replace(np.nan, np.inf)

range_frame['Min'] = range_frame['Min'].astype(float)
range_frame['Max'] = range_frame['Max'].astype(float)

def conditions(merged):
  for i in range(number_of_bins):
    if ((range_frame.iloc[i, 0] <= merged["Ratios: HLG / ALL (Whole)"]) & (merged["Ratios: HLG / ALL (Whole)"] <= range_frame.iloc[i, 1])):
      return i + 1
    if ((range_frame.iloc[i, 0] <= merged["Ratios: HLG / ALL (Whole)"]) & (range_frame.iloc[i,1] == np.inf)):
      return number_of_bins + 1

merged["Bin Type"] = merged.apply(conditions, axis = 1)

merged = merged[['Gene', "Confidence Level", "Ratios: HLG / ALL (Whole)", "Bin Type"]]
# reorder merged 
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index
merged["Bin Type"] = merged["Bin Type"].astype(int)

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library:
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

unique_gene = pd.DataFrame(clean['Gene'].unique())                              # dataframe to store the unique gene names
bin_type = clean[['Gene', "Bin Type"]]                                          # dataframe to store all genes + mutants + bin type

super_bins = []                                                                 # declare empty list
xl_p_val = []                                                                   # declare empty list

for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = bin_type.loc[bin_type.Gene == unique_gene.iloc[i,0]]              # find where the unique gene name dataframe at i equals the bin_type dataframe with all the mutants
  bin_df = pd.DataFrame(same_gene["Bin Type"].value_counts()).reindex(index = range(1,number_of_bins + 1)).transpose().fillna(0).astype(int)
  # splice by gene names, count occurrence of every bin number, make columns from 1 - ... for the bin numbers, replace 'NaN' with 0, change from float to integer, switch x and y axis

  bin_df = bin_df.set_index(same_gene['Gene'].unique())                         # make the unique gene names the index

  super_bins.append(bin_df)                                                     # append data to each other, avoid long run times
  # looks like: [Total in: 1 , Total in: 2 , Total in: 3 , Total in: 4 , etc.]
# end of for loop


super_bins = pd.concat(super_bins, axis = 0)                                    # concatenate the lists into a dataframe
super_bins = super_bins.transpose()                                             # switch x and y axis again
super_bins["Total"] = super_bins.sum(axis = 1)                                  # add up total count in each bin to create the 'Total' 
super_bins = super_bins.transpose()                                             # switch x and y axis again

total = super_bins.loc[super_bins.index == 'Total']                             # make total dataframe, and the index name to be 'Total'


for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = super_bins.loc[super_bins.index == unique_gene.iloc[i,0]]         # find where the unique gene name dataframe at i equals the bin_type dataframe with all the mutants
  
  bin_frames = [same_gene, total]                                               # create a contingency table, 'frame' with the the individual gene bin table and the total
  bin_concat = pd.concat(bin_frames)                                            # concatenate bin_frames ↑
  m = np.array(bin_concat)                                                      # change from dataframe to an array  
  res = stats.fisher_test(m, workspace = 2e+06)                                 # do the r fisher test, and expand workspace since the default, 2+e05 was too small         
  val = format(res[0][0])                                                       # re-format the results

  xl_p_val.append(val)                                                          # append the results to empty list
# end of for loop

xl_p_val = pd.DataFrame(xl_p_val, columns = ['p-value'])                        # convert list to dataframe with the column, 'p-value'
xl_p_val = pd.concat([unique_gene, xl_p_val], axis = 1).rename(columns = {0 : 'Gene'}).set_index('Gene')
# concatenate the the unique gene names to the p-values, side by side, rename the column, set the index of the dataframe to be the gene name

super_bins = super_bins.reset_index().rename(columns = {'index' : 'Gene'})      # reset the index of super_bins and rename column
xl_p_val = xl_p_val.reset_index()                                               # reset index of xl_p_val

bin_and_p_val = pd.merge(super_bins, xl_p_val, on = 'Gene').set_index('Gene')   # merge super_bins and xl_p_val on the 'Gene' column and set the index to be the gene names
bin_and_p_val["Adjusted p-value"] = FDR_adjust_pvalues(bin_and_p_val["p-value"])

display(bin_and_p_val.sort_values("Adjusted p-value"), total)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of bins: 4
Drop data if the total counts of a gene is less than: 3
Drop data if the confidence level is greater than: 2

Enter the range signs as you would reading ranges left to right, keep in mind possible overlapping ranges
Bin Number 1 :
	Minimum: 0.0
	Maximum: 0.5
Enter sign for the right-hand side of the inequality: <=
0.0 ≤ x ≤ 0.5
Bin Number 2 :
	Minimum: 0.5
	Maximum: 1.0
Enter sign for the left-hand side of the inequality: <
Enter sign for the right-hand side of the inequality: <=
0.5 < x ≤ 1.0
Bin Number 3 :
	Minimum: 1.0
	Maximum: 2.0
Enter sign for the left-hand side of the inequality: <
Enter sign for the right-hand side of the inequality: <=
1.0 < x ≤ 2.0
Bin Number 4 :
	Minimum: 2.0
Enter sign for the inequality: <
x < 2.0


bin_range_1  :  0.0 <= x <= 0.5
bin_range_2  :  0.5 < x <= 1.0
bin_range_3  :  1.0 < x <= 2.0
bin_range_4  :  x < 2.0


,1,2,3,4,p-value,Adjusted p-value
Gene,,,,,,
Cre01.g000017,4,0,1,0,0.39087615692192684,1.0
Cre12.g486208,4,0,1,0,0.39087615692192684,1.0
Cre12.g486100,3,0,0,0,0.9999999998352762,1.0
Cre12.g486000,2,0,1,0,0.3811793836588967,1.0
Cre12.g485950,3,0,1,1,0.4819080265349122,1.0
...,...,...,...,...,...,...
Cre06.g266383,2,0,0,1,0.6640813441211517,1.0
Cre06.g266350,13,0,0,1,0.5072669358335904,1.0
Cre06.g266300,9,1,0,3,0.5911809457339393,1.0


,1,2,3,4
Total,41343,2396,4128,11606


# Bin p-values, hard-coded version

In [ ]:
import numpy as np
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()

stats = importr('stats')

tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
print("1. Ratio of 0.0 ≤ x ≤ 0.5\n2. Ratio of 0.5 < x ≤ 1.0\n3. Ratio of 1.0 < x ≤ 2.0\n4. Ratio of 2.0+")
#!
# user input for choice of bin
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns

def conditions(merged):
  if ((merged["Ratios: HLG / ALL (Whole)"] >= 0.0) & (merged["Ratios: HLG / ALL (Whole)"] <= 0.5)):
    return 1
  elif ((merged["Ratios: HLG / ALL (Whole)"] > 0.5) & (merged["Ratios: HLG / ALL (Whole)"] <= 1.0)):
    return 2
  elif ((merged["Ratios: HLG / ALL (Whole)"] > 1.0) & (merged["Ratios: HLG / ALL (Whole)"] <= 2.0)):
    return 3
  elif (merged["Ratios: HLG / ALL (Whole)"] > 2.0):
    return 4
    
merged["Bin Type"] = merged.apply(conditions, axis = 1)                         # new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'

merged = merged[['Gene', "Confidence Level", "Ratios: HLG / ALL (Whole)", "Bin Type"]]
# reorder merged 
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library:
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

unique_gene = pd.DataFrame(clean['Gene'].unique())                              # dataframe to store the unique gene names
bin_type = clean[['Gene', "Bin Type"]]                                          # dataframe to store all genes + mutants + bin type

super_bins = []                                                                 # declare empty list
xl_p_val = []                                                                   # declare empty list

for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = bin_type.loc[bin_type.Gene == unique_gene.iloc[i,0]]              # find where the unique gene name dataframe at i equals the bin_type dataframe with all the mutants
  bin_df = pd.DataFrame(same_gene["Bin Type"].value_counts()).reindex(index = range(1,5)).transpose().fillna(0).astype(int)
  # splice by gene names, count occurrence of every bin number, make columns from 1 - 5 for the bin numbers, replace 'NaN' with 0, change from float to integer, switch x and y axis

  bin_df = bin_df.set_index(same_gene['Gene'].unique())                         # make the unique gene names the index

  super_bins.append(bin_df)                                                     # append data to each other, avoid long run times
  # looks like: [ Total in: 1 , Total in: 2 , Total in: 3 , Total in: 4]
# end of for loop

super_bins = pd.concat(super_bins, axis = 0)                                    # concatenate the lists into a dataframe
super_bins = super_bins.transpose()                                             # switch x and y axis again
super_bins["Total"] = super_bins.sum(axis = 1)                                  # add up total count in each bin to create the 'Total' 
super_bins = super_bins.transpose()                                             # switch x and y axis again

total = super_bins.loc[super_bins.index == 'Total']                             # make total dataframe, and the index name to be 'Total'


for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = super_bins.loc[super_bins.index == unique_gene.iloc[i,0]]         # find where the unique gene name dataframe at i equals the bin_type dataframe with all the mutants
  
  bin_frames = [same_gene, total]                                               # create a contingency table, 'frame' with the the individual gene bin table and the total
  bin_concat = pd.concat(bin_frames)                                            # concatenate bin_frames ↑
  m = np.array(bin_concat)                                                      # change from dataframe to an array  
  res = stats.fisher_test(m, workspace = 2e+06)                                 # do the r fisher test, and expand workspace since the default, 2+e05 was too small         
  val = format(res[0][0])                                                       # re-formate the results

  xl_p_val.append(val)                                                          # append the results to empty list
# end of for loop

xl_p_val = pd.DataFrame(xl_p_val, columns = ['p-value'])                        # convert list to dataframe with the column, 'p-value'
xl_p_val = pd.concat([unique_gene, xl_p_val], axis = 1).rename(columns = {0 : 'Gene'}).set_index('Gene')
# concatenate the the unique gene names to the p-values, side by side, rename the column, set the index of the dataframe to be the gene name

super_bins = super_bins.reset_index().rename(columns = {'index' : 'Gene'})      # reset the index of super_bins and rename column
xl_p_val = xl_p_val.reset_index()                                               # reset index of xl_p_val

bin_and_p_val = pd.merge(super_bins, xl_p_val, on = 'Gene').set_index('Gene')   # merge super_bins and xl_p_val on the 'Gene' column and set the index to be the gene names
bin_and_p_val["Adjusted p-value"] = FDR_adjust_pvalues(bin_and_p_val["p-value"])

display(bin_and_p_val.sort_values("Adjusted p-value"), total)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Drop data if the total counts of a gene is less than: 3
Drop data if the confidence level is greater than: 2
1. Ratio of 0.0 ≤ x ≤ 0.5
2. Ratio of 0.5 < x ≤ 1.0
3. Ratio of 1.0 < x ≤ 2.0
4. Ratio of 2.0+


,1,2,3,4,p-value,Adjusted p-value
Gene,,,,,,
Cre01.g000017,4,0,1,0,0.39087615692192684,1.0
Cre12.g486208,4,0,1,0,0.39087615692192684,1.0
Cre12.g486100,3,0,0,0,0.9999999998352762,1.0
Cre12.g486000,2,0,1,0,0.3811793836588967,1.0
Cre12.g485950,3,0,1,1,0.4819080265349122,1.0
...,...,...,...,...,...,...
Cre06.g266383,2,0,0,1,0.6640813441211517,1.0
Cre06.g266350,13,0,0,1,0.5072669358335904,1.0
Cre06.g266300,9,1,0,3,0.5911809457339393,1.0


,1,2,3,4
Total,41343,2396,4128,11606


# Choose bin



In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level

bin_choice = float(input("1. Ratio of 0.0 ≤ x ≤ 0.5\n2. Ratio of 0.5 < x ≤ 1.0\n3. Ratio of 1.0 < x ≤ 2.0\n4. Ratio of 2.0+\nChoose Bin: "))
#!
# user input for choice of bin
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns

if ((bin_choice < 1) | (bin_choice > 4)):
  print("\nInvalid input")
  sys.exit()

elif bin_choice == 1:
  bin_result = merged.loc[((merged["Ratios: HLG / ALL (Whole)"] >= 0.0) & (merged["Ratios: HLG / ALL (Whole)"] <= 0.5))]

elif bin_choice == 2:
  bin_result = merged.loc[((merged["Ratios: HLG / ALL (Whole)"] > 0.5) & (merged["Ratios: HLG / ALL (Whole)"] <= 1.0))]

elif bin_choice == 3:
  bin_result = merged.loc[((merged["Ratios: HLG / ALL (Whole)"] > 1.0) & (merged["Ratios: HLG / ALL (Whole)"] <= 2.0))]

elif bin_choice == 4:
  bin_result = merged.loc[(merged["Ratios: HLG / ALL (Whole)"] > 2.0)]

# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'

bin_result = bin_result[['Gene', "Confidence Level", "Ratios: HLG / ALL (Whole)"]]
# reorder merged 
bin_result = bin_result.sort_values('Gene')
# sort the values by the gene name
bin_result = bin_result.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = bin_result['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = bin_result['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = bin_result.merge(merged_count_drop, on = "Gene")                  # merge merged and merged_count_drop based on gene name

clean
